In [1]:
import os
import cv2
import numpy as np


In [10]:
original_dataset_path = r"D:\MED_LEAF_ID\dataset\Medicinal Leaf dataset"
augmented_dataset_path = r"D:\MED_LEAF_ID\data\augmented"
preprocessed_original_path = r"D:\MED_LEAF_ID\data\cnn\original"
preprocessed_augmented_path = r"D:\MED_LEAF_ID\data\cnn\augmented"

In [11]:
def create_directory_structure(dataset_path, preprocessed_path):
    os.makedirs(preprocessed_path, exist_ok=True)
    for class_folder in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_folder)
        if os.path.isdir(class_path):
            os.makedirs(os.path.join(preprocessed_path, class_folder), exist_ok=True)


In [12]:
def histogram_equalization(image):
    if len(image.shape) == 3:  # Color Image
        ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
        ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])  # Equalize Luminance
        return cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)
    else:  # Grayscale Image
        return cv2.equalizeHist(image)

In [13]:
def global_contrast_normalization(image, scale=1.0, epsilon=1e-8):
    mean = np.mean(image)
    std = np.std(image)
    return scale * (image - mean) / max(std, epsilon)


In [14]:
def local_normalization(image, ksize=7):
    blurred = cv2.GaussianBlur(image, (ksize, ksize), 0)
    return cv2.subtract(image, blurred)

In [15]:
def preprocess_image_for_cnn(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return None
    
    # Resize to 224x224
    resized_image = cv2.resize(image, (224, 224))

    # Apply Histogram Equalization
    enhanced_image = histogram_equalization(resized_image)

    # Apply Global Contrast Normalization
    gcn_image = global_contrast_normalization(enhanced_image)

    # Apply Local Normalization
    normalized_image = local_normalization(gcn_image)

    # Convert to Float32 and Normalize to [0,1]
    normalized_image = normalized_image.astype('float32') / 255.0

    return normalized_image


In [16]:
def preprocess_and_save_images(dataset_path, preprocessed_path):
    for class_folder in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_folder)
        if os.path.isdir(class_path):
            save_path = os.path.join(preprocessed_path, class_folder)
            for image_file in os.listdir(class_path):
                image_path = os.path.join(class_path, image_file)
                processed_image = preprocess_image_for_cnn(image_path)
                if processed_image is not None:
                    cv2.imwrite(os.path.join(save_path, image_file), (processed_image * 255).astype(np.uint8))
                else:
                    print(f"Skipping image: {image_file}")


In [17]:
if __name__ == "__main__":
    create_directory_structure(original_dataset_path, preprocessed_original_path)
    create_directory_structure(augmented_dataset_path, preprocessed_augmented_path)
    preprocess_and_save_images(original_dataset_path, preprocessed_original_path)
    preprocess_and_save_images(augmented_dataset_path, preprocessed_augmented_path)
    
    print("Preprocessing complete with Contrast Normalization & Edge Enhancement!")

Preprocessing complete with Contrast Normalization & Edge Enhancement!
